In [ ]:
from IPython.core.display import HTML
display(HTML('<style>.container { width:95% !important; } </style>'))

%load_ext autoreload
%autoreload 2

# Practical Session - Planning in bandits/MDP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from tqdm.notebook import tqdm

from forban import *
from forban.bandits import NormalBandit, Normal
from forban.sequentialg import SequentiAlg
from forban.utils import *

# Part I - Best Arm Identification

# Bandit environment, how to?

## Distributions - Arms

In [ ]:
mean_1 = 0.
mean_2 = 1.
normal_1 = Normal(mean_1)
normal_2 = Normal(mean_2)
sample_1 = normal_1.sample()
sample_2 = normal_2.sample()
means = [mean_1, mean_2]
print(f"A sample from a normal distribution with mean {mean_1} and standard deviation 1 is {sample_1:.3f}")
print(f"A sample from a normal distribution with mean {mean_2} and standard deviation 1 is {sample_2:.3f}")

## Bandit instance

In [ ]:
bandit_instance = NormalBandit([0.1, 0.2, 0.32, 0.24, 0.22])
print(f"A bandit instance is {bandit_instance}")

In [ ]:
plot_bandit(bandit_instance)

## SequentiAlg class

In [ ]:
alg = SequentiAlg(bandit_instance)
print(alg)

In [ ]:
# Play arm 0 ...
alg.pull(0)
print(alg)

# ... then play arm 1
alg.pull(1)
print(alg)

In [ ]:
print('time: ', alg.time)
print('Number of pulls: ', alg.nbr_pulls)
print('Empirical means: ', alg.means)

In [ ]:
# Run the algorithm for 1000 steps.
# By default, the choose_an_arm method implements uniform random selection among arms.
# You will implement other selection methods in the following.
alg.fit(1000)
print(alg)

### Question 1

What are the three main parts of an exploration algorithm that aims to identify the best arm (*i.e.*, the arm having the largest mean)?

### Exploration class template

In [ ]:
class Exploration(SequentiAlg):
    def __init__(self, bandit, name="Exploration", params={'init': 0, 'stopping criterion': None }):
        SequentiAlg.__init__(self, bandit, name=name, params=params)
        self.stopping_criterion = params.get('stopping criterion', None)
        
    def compute_indices(self):
        pass
            
    def choose_an_arm(self):
        return randamax(self.indices)
    
    def sampling(self):
        _ = self.pull(self.choose_an_arm())
    
    def recommendation(self):
        pass
    
    def stop(self):
        pass
    
    def run(self):
        while not self.stop():
            self.sampling()
            self.recommendation()

## Template for doing experiments

In [ ]:
class Experiment:
    def __init__(self, sequential_algorithms, bandit,
                 statistics={'distance':True, 'proba':True, 'pulls':True}):
        assert len(sequential_algorithms) > 0
        self.algorithms = sequential_algorithms
        self.bandit = bandit
        self.nbr_algo = len(sequential_algorithms)
        self.algo_idx = 0
        self.statistics = {}
        self.nbr_exp = None
        for algo_idx, algo in enumerate(self.algorithms):
            self.statistics[algo_idx] = {'name': algo.name}
            for s in statistics.items():
                if s[1]:
                    self.statistics[algo_idx][s[0]] = None
        stats = self.statistics[0].keys()
        self.stats = stats

    def run(self, nbr_exp=500):
        self.nbr_exp = nbr_exp
        for algo_idx, algo in enumerate(self.algorithms):
            reco_arms = np.zeros(nbr_exp+self.bandit.nbr_arms, int)
            delta = np.zeros(nbr_exp+self.bandit.nbr_arms)
            pulls = np.zeros(nbr_exp, int)
            for i in tqdm(range(nbr_exp)):
                algo.reset()
                algo.run()
                
                reco_arms[i] = algo.recommendation()
                delta[i] = self.bandit.regrets[reco_arms[i]]
                pulls[i] = np.sum(algo.nbr_pulls)
            
            reco_arms[nbr_exp:] = np.arange(self.bandit.nbr_arms)
            delta[nbr_exp:] = self.bandit.regrets
            for k in self.stats:
                if k == 'distance':
                    self.statistics[algo_idx][k] = delta.copy()
                elif k == 'proba':
                    self.statistics[algo_idx][k] = reco_arms.copy()
                elif k == 'pulls':
                    self.statistics[algo_idx]['pulls'] = pulls.copy()
                    
    def plot(self, normalisation=False):
        exp_info = f"Nbr. of experiments = {self.nbr_exp}"
        ticks = np.arange(self.bandit.nbr_arms)
        if 'proba' in self.stats:
            plt.figure(figsize=(12,5))
            title = "Histogram of recommendation\n"
            title += exp_info
            plt.title(title)
            plt.xlabel("Arm", fontsize=14)
            plt.xticks(ticks, [str(i) for i in range(self.bandit.nbr_arms)])
            
            n = self.nbr_algo
            w = 1/(2*n)
            
            for i, algo_stats in enumerate(self.statistics.values()):
                name = algo_stats['name']
                _, hist = np.unique(algo_stats['proba'], return_counts=True)
                hist = hist - np.ones(self.bandit.nbr_arms, int)
                if normalisation:
                    hist = hist / self.nbr_exp
                    ax = plt.gca()
                    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
                t = ticks + w*(i - (n-1)/2)
                plt.bar(t, hist, width=w, label=name, alpha=0.5)
    
            plt.legend()
            plt.show()
        
        ticks = np.sort(self.bandit.regrets)
        if 'distance' in self.stats:
            plt.figure(figsize=(12,5))
            title = "Histogram of recommendation regret\n"
            title += exp_info
            plt.title(title)
            plt.xlabel("Arm - regret", fontsize=14)
            idxs = np.argsort(self.bandit.regrets)
            tmp = self.bandit.regrets[idxs]
            tmp = np.abs(tmp[1:] - tmp[:-1])
            w = np.unique(tmp)
            w = w[w > 0]
            
            n = self.nbr_algo
            w = w[0] / (2*n)
            
            plt.xticks(ticks, [f"{idxs[i]} - {self.bandit.regrets[idxs[i]]:.3f}" for i in range(self.bandit.nbr_arms)])

            for i, algo_stats in enumerate(self.statistics.values()):
                name = algo_stats['name']
                delta = algo_stats['distance']
                _, hist = np.unique(delta, return_counts=True)
                hist = hist - np.ones(self.bandit.nbr_arms, int)
                if normalisation:
                    hist = hist / self.nbr_exp
                    ax = plt.gca()
                    ax.yaxis.set_major_formatter(FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
                t = ticks + w*(i - (n-1)/2)
                plt.bar(t, hist, width=w, label=name, alpha=0.5)
    
            plt.legend()
            plt.show()
            
        if 'pulls' in self.stats:
            plt.figure(figsize=(12,5))
            title = "Histogram of number of pulls\n"
            title += exp_info
            plt.title(title)
            plt.xlabel("Number of pulls", fontsize=14)
            
            w = np.max([np.max(algo_stats['pulls']) for algo_stats in self.statistics.values()]) / 12
            
            color = plt.cm.rainbow(np.linspace(0, 1, self.nbr_algo))
            
            for i, algo_stats in enumerate(self.statistics.values()):
                name = algo_stats['name']
                p = plt.hist(algo_stats['pulls'], width=w, label=name, alpha=0.5, bins=10, color=color[i])
                plt.axvline(x=np.mean(algo_stats['pulls']), linestyle='dashed', label=name + ' mean pulls', color=color[i])
            plt.legend()
            plt.show()

## Warm-up: uniform exploration

### Question 2
Using the `Exploration` class, implement an exploration algorithm with uniform allocation. Test different recommendation rules on at least two different bandit problems.

In [ ]:
bandit_instance = # TODO

In [ ]:
class ExplorationUniform(Exploration):
    def __init__(self, bandit, name="Exploration", params={'init': 0, 'stopping criterion': 500 }):
        Exploration.__init__(self, bandit, name=name, params=params)
        self.stopping_criterion = params['stopping criterion']
        
    def compute_indices(self):
        # TODO
            
    def choose_an_arm(self):
        return randamax(self.indices)
    
    def sampling(self):
        _ = self.pull(self.choose_an_arm())
    
    def recommendation(self):
        # TODO
    
    def stop(self):
        # TODO
    
    def run(self):
        while not self.stop():
            self.sampling()
            self.recommendation()

In [ ]:
uniform_1 = ExplorationUniform(
    bandit_instance, 
    name="Exploration 200",
    params={'init': 0, 'stopping criterion': 200}
)

In [ ]:
expe = Experiment(
    [uniform_1],  # add more algorithms?
    bandit_instance
)

In [ ]:
expe.run(200)

In [ ]:
expe.plot(True)

## Fixed budget setting

### Question 3

Recall what the UCB algorithm is for cumulative regret. Derive a **sampling rule** for best arm identification inspired by UCB.

Using the `Exploration` class, implement this idenfication algorithm. Test different **recommendation rules** on different bandit problems.

In [ ]:
class ExplorationUCB(Exploration):
    def __init__(self, bandit, name="ExplorationUCB", params={'init': 0, 'stopping criterion': 500}):
        Exploration.__init__(self, bandit, name=name, params=params)
        self.stopping_criterion = params['stopping criterion']
        
    def compute_indices(self):
        # TODO

    def choose_an_arm(self):
        return randamax(self.indices)
    
    def sampling(self):
        _ = self.pull(self.choose_an_arm())
    
    def recommendation(self):
        # TODO
        
    def stop(self):
        # TODO
    
    def run(self):
        while not self.stop(): 
            self.sampling()
            self.recommendation()

In [ ]:
ucb_1 = ExplorationUCB(
    bandit_instance, 
    name="ExplorationUCB_1", 
    params={
        'init': 0, 
        'stopping criterion': 500, 
        # other parameters?
    }
)


In [ ]:
expe = Experiment(
    [ucb_1],  # add more algorithms?
    bandit_instance
)

In [ ]:
expe.run(200)

In [ ]:
expe.plot(True)

### Question 4.1

Relate the *simple regret* of the exploration strategy UCB+Empirical Distribution of Plays to the *cumulative regret* of UCB.

### Question 4.2

How good is UCB? How does it compare to uniform exploration? What is the influence of the various parameters? Compare the different recommendation rule.

### Question 5
Using the `Exploration` class, implement the sequential halving algorithm with a fixed budget. Test it on different bandit problems.

In [ ]:
class ExplorationHalf(Exploration):
    def __init__(self, bandit, name="ExplorationHalf", params={'init': 0, 'stopping criterion': 500}):
        Exploration.__init__(self, bandit, name=name, params=params)
        self.stopping_criterion = params['stopping criterion']
        self.nbr_arms = self.bandit.nbr_arms

    def sampling(self):
        # TODO
    
    def recommendation(self):
        # TODO
    
    def stop(self):
        # TODO
    
    def run(self):
        while not self.stop(): 
            self.sampling()
            self.recommendation()

In [ ]:
sequential_halving_1 = ExplorationHalf(
    bandit_instance, 
    name="ExplorationHalf_1", 
    params={
        'init': 0, 
        'stopping criterion': 500,
    }
)

In [ ]:
expe = Experiment([sequential_halving_1, sequential_halving_2], bandit_instance)

In [ ]:
expe.run(200)

In [ ]:
expe.plot(True)

### Question 6

How good is sequential halving? Cite at least one drawback compared to UCB.

## Fixed confidence setting

Fixed confidence algorithms return a best arm recommendation after a **variable** number of steps $\tau_{\delta}$, so that the probability that the recommendation is correct is at least $1-\delta$.

We will derive such an algorithm in the case of $\textbf{K=2}$ arms with Gaussian distributions $\mathcal{N}(\mu_1, \sigma^2)$ and $\mathcal{N}(\mu_2, \sigma^2)$ ($\sigma$ is known), in which case all calculations are explicit.

Let $\mu = \begin{pmatrix} \mu_1 \\ \mu_2\end{pmatrix}\in\mathbb{R}^2$ the vector of means. Assume for simplicity that $1$ is the optimal arm, i.e $\mu_1 > \mu_2$. We define the set of *alternatives* to $\mu$ as $\text{Alt}(\mu) = \{ \lambda\in\mathbb{R}^2, \lambda_1 < \lambda_2\}$, i.e the set of mean vectors such that in the bandit they represent, arm $1$ is *not* optimal.

An important quantity to analyze best-arm-identification algorithms is the **characteristic time**:
$$
T_*(\mu)^{-1} = \max_{w\in[0, 1]} \inf_{\lambda\in\text{Alt}(\mu)} w \text{kl}(\mu_1, \lambda_1) + (1-w) \text{kl}(\mu_2, \lambda_2),
$$
where $\text{kl}(\mu_k, \lambda_k)$ is Kullback-Leibler divergence between $\mathcal{N}(\mu_k, \sigma^2)$ and $\mathcal{N}(\lambda_k, \sigma^2)$.

### Question 7.1
Recall the expression of the KL divergence $kl(\mu_k, \lambda_k)$. Have you seen the quantity $T_*(\mu)$ in class?

Let's have a look at the $w\in[0, 1]$ for which the maximum is attained in the definition of $T_*(\mu)$. 

The vector $\begin{pmatrix}w_*(\mu)\\1-w_*(\mu)\end{pmatrix}$ is called the vector of **optimal allocation**. In theory, a *good* identification algorithm should select arm $1$ around $w_*(\mu)$ fraction of the time, and arm $2$ around $1-w_*(\mu)$ fraction of the time. 

### Question 7.2 Compute the optimal allocation

For $w\in(0,1)$, let $J(\lambda)= w (\mu_1 - \lambda_1)^2 + (1-w) (\mu_2 - \lambda_2)^2$. Write the Lagrangian associated with the (convex) problem
$$
\min_{\lambda_1\leq \lambda_2} J(\lambda).
$$
Explain why the linear constraint $\lambda_1\leq \lambda_2$ must be saturated. Find the value of the minimization problem.

Finally, explain how this problem is related to the characteristic time and compute the vector of optimal allocation (you may assume $w_*(\mu)=0$ and $w_*(\mu)=1$ are prohibited, i.e the optimal allocation needs to put some mass on both arm -- this is quite intuitive but a bit tedious to show).

N.B: in the general case $K>2$ and for other distributions than Gaussian, the expression of the optimal allocation is not available in closed-form. However, it is usually easy to numerically approximate it with convex programming solvers.

Track-and-Stop implements the three components of a pure exploration algorithm as follows.

Let $\hat{\mu}_{k,t}$ the empirical mean of arm $k$ after $t$ steps and $\hat{\mu}_t=\begin{pmatrix}\hat{\mu}_{1,t}\\ \hat{\mu}_{2,t}\end{pmatrix}$.

* Sampling rule: at $t+1$, the policy is 
$$
\pi_{t+1}\in \begin{cases}
\textbf{Forced exploration:}\quad \arg\min_{k\in \{ 1, \dots, K\}} N_k(t) & \text{if $N_k(t) < \sqrt{t}$ for some $k\in\{1, \dots, K\}$}\\
\quad\textbf{Tracking:}\quad \arg\max_{k\in \{ 1, \dots, K\}} w_*(\hat{\mu}_t)_k - \frac{N_k(t)}{t} & \text{else}.
\end{cases}
$$

* Stopping rule: let 
    * $Z_t = \inf_{\lambda\in\text{Alt}(\hat{\mu}_t)} \sum_{k=1}^K N_k(t) \text{kl}(\hat{\mu}_{k,t}, \lambda_k)$,
    * $\beta(t, \delta)$: some function that grows like $\log\log(t)$.
    
    The algorithm stops after $\tau_\delta$ steps, where
$$
\tau_{\delta} = \inf\{ t\in\mathbb{N}, Z_t > \beta(t, \delta) \}.
$$

* Recommendation rule: best empirical mean after stopping: $\arg\max_{k\in\{1, \dots, K\}} \hat{\mu}_{k, \tau_{\delta}}$.

### Question 7.3
Using the results of 5.2, write the Track-and-Stop algorithm in the Gaussian case (same known variance) with $K=2$ arms. Comment on the sampling rule.

### Question 8

Using the `Exploration` class, implement the Track-and-Stop algorithm in the setting of Question 7. 

You may use $\beta(t, \delta) = \log\frac{1 + \log t}{\delta}$.

Test it on different bandit problems.

In [ ]:
class ExplorationTaS(Exploration):
    def __init__(self, bandit, name="ExplorationTaS", params={'init': 0}):
        Exploration.__init__(self, bandit, name=name, params=params)
        self.nbr_arms = self.bandit.nbr_arms
        assert self.nbr_arms == 2, "We only do 2 arms!"
    
    def compute_indices(self):
        # TODO

    def choose_an_arm(self):
        return randamax(self.indices)
    
    def sampling(self):
        _ = self.pull(self.choose_an_arm())
        
    def recommendation(self):
        # TODO

    def stop(self):
        # TODO
    
    def run(self):
        while not self.stop(): 
            self.sampling()
            self.recommendation()

In [ ]:
sigma = 1.0
bandit_instance = NormalBandit([0.25, 0.1], stds=[sigma, sigma])
plot_bandit(bandit_instance)

In [ ]:
delta = 0.1

track_and_stop = ExplorationTaS(
    bandit_instance, 
    name="ExplorationTaS", 
    params={
        'init': 0,
        # other parameters?
    }
)

In [ ]:
expe = Experiment(
    [track_and_stop], # more algorithms?
    bandit_instance
)

In [ ]:
expe.run(200)

In [ ]:
expe.plot(True)

## Question 9

Using the characteristic time $T_*(\mu)$, compute a lower bound on the sampling complexity $\mathbb{E}[\tau_\delta]$. 

Comment on the performances of Track-and-Stop. Do you know if the sample complexity of Track-and-Stop matches this lower bound? Is it what you observe in practice? If not, can you think of why?

Feel free to try different bandit problems as well.

### Question 10
Do you think using structure would help for BAI?